# 4. 贝叶斯分类器

计算后验概率 $P(c|x)$大体有两种策略：给定x，可通过直接建模 $P(c|x)$来预测c，这样得到的是“判别式模型”(discriminative models)；也可先对联合概率分布$P(x|c)$建模，然后再由此获得$P(c|x)$，这样得到的是“生成式模型”(generative models)。决策树，BP神经网络，支持向量机等，都可归入到判别式模型的范畴，对生成式模型来说 $P(c|x)=\frac{P(x,c)}{P(x)}$，基于贝叶斯定理，$P(c|x)$可以写为  
$P(c|x) = \frac{P(c)P(x|c)}{P(x)}$  

其中P(c)是类先验概率；P(x|c)是样本x在类标记c的条件下的条件概率，或称为“似然”；P(x)是用于归一化的“证据因子”。对给定样本x，证据因子P(x)与类标记无关，因此估计$P(c|x)$的问题就转化成为如何基于训练数据D来估计先验$P(c)$和似然$P(x|c)$

基于贝叶斯公式来估计后验概率$P(c|x)$的主要困难在于：类条件概率$P(x|c)$是所有属性上的联合概率，难以从有限的训练样本直接估计而得。为避开这个障碍，朴素贝叶斯分类器采用了“属性条件独立性假设”对一直类别，假设所有属性相互独立。换言之，假设每个属性独立地对分类结果发生影响。基于属性条件独立性假设，贝叶斯公式可重写为：

$P(c|x)=\frac{P(c)P(x|c)}{P(x)}=\frac{P(c)}{P(x)}\prod_{i=1}^dP(x^i|c)$

其中d为属性数目，$x_i$为x在第i个属性上的取值。

朴素贝叶斯分类器的训练过程就是基于训练集D来估计类先验概率P(c)，并为每个属性估计条件概率$P(x_i|c)$

令$D_c$表示训练集D中第c类样本组成的集合，若有充足的独立同分布样本，则可容易地估计出先验概率$P(c)=\frac{|D_C|}{|D|}$，

对离散属性而言，令$D_{c,x_i}$表示$D_c$中在第i个属性上取值为$x_i$的样本组成的集合，则条件概率$P(x_i|c)$可估计为$P(x_i|c)=\frac{|D_{c,x_i}|}{|D_c|}$

对连续属性可考虑概率密度函数，假定$p(x_i|c)\approx N(\mu_{c,i},\sigma_{c,i}^2)$，其中$\mu_{c,i}$和$\sigma_{c,i}^2$分别是第c类样本在第i个属性上取值的均值和方差，则有  
$
   p(x_i|c)=\frac{1}{\sqrt{2\pi}\sigma_{c,i}} exp(-\frac{(x_i-\mu_{c,i})^2}{2\sigma_{c,i}^2})
$

## 4.1. 以西瓜数据为例

In [1]:
import pandas as pd
df = pd.read_csv("../chp03-decisionTree/watermallon.bak")
df

,color,root,knock,stripp,sto,feel,labels
0,青绿,蜷缩,浊响,清晰,凹陷,硬滑,good
1,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,good
2,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,good
3,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,good
4,浅白,蜷缩,浊响,清晰,凹陷,硬滑,good
5,青绿,稍蜷,浊响,清晰,稍凹,软粘,good
6,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,good
7,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,good
8,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,bad
9,青绿,硬挺,清脆,清晰,平坦,软粘,bad


对于测试样例：

|  编号   |  色泽  |  根蒂  |  敲声  | 纹理  |  脐部  |  触感  |  好瓜 |
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|  1  |  青绿  |  蜷缩  |  浊响  |  清晰 |  凹陷  |  硬滑  |  ？ |

首先估计先验概率P(c)，显然有：

$ 
        P(好瓜=是)=\frac{8}{17}\approx0.471 \\
        P(好瓜=否)=\frac{9}{17}\approx0.529 \\
$

然后，为每个属性估计条件概率：

$
P(青绿|是)=P(色泽=青绿|好瓜=是)=\frac{3}{8}=0.375 \\
P(青绿|否)=P(色泽=青绿|好瓜=否)=\frac{3}{9}\approx0.333 \\
P(蜷缩|是)=P(根蒂=蜷缩|好瓜=是)=\frac{5}{8}=0.625 \\
P(蜷缩|否)=P(根蒂=蜷缩|好瓜=否)=\frac{3}{9}\approx0.333 \\
P(浊响|是)=P(敲声=浊响|好瓜=是)=\frac{6}{8}=0.750 \\
P(浊响|否)=P(敲声=浊响|好瓜=否)=\frac{4}{9}\approx0.444 \\
P(清晰|是)=P(纹理=清晰|好瓜=是)=\frac{7}{8}=0.875 \\
P(清晰|否)=P(纹理=清晰|好瓜=否)=\frac{2}{9}\approx0.222 \\
P(凹陷|是)=P(脐部=凹陷|好瓜=是)=\frac{6}{8}=0.750 \\
P(凹陷|否)=P(脐部=凹陷|好瓜=否)=\frac{2}{9}\approx0.222 \\
P(硬滑|是)=P(触感=硬滑|好瓜=是)=\frac{6}{8}=0.750 \\
P(硬滑|否)=P(触感=硬滑|好瓜=否)=\frac{6}{9}\approx0.667 \\
$

于是有
 
$
P(好瓜=是)×P(青绿|是)*P(蜷缩|是)*P(浊响|是)*P(清晰|是)*P(凹陷|是)*P(硬滑|是)\approx0.0407  \\
P(好瓜=否)×P(青绿|否)*P(蜷缩|否)*P(浊响|否)*P(清晰|否)*P(凹陷|否)*P(硬滑|否)\approx0.0008  \\
$

因为0.047>0.0008，所以，朴素贝叶斯分类器将测试样本判别为”好瓜“。

In [2]:
0.471*0.375*0.625*0.750*0.875*0.750*0.750

0.04074966430664063

In [3]:
0.529*0.333*0.333*0.444*0.222*0.222*0.667

0.0008561678034527272

**注意**

若某个属性值在训练集中没有与某个类同时出现过，则会出现问题，连乘式计算的概率为0.为了避免其他属性携带的信息被训练集中未出现的属性值”抹去“，在估计概率值时通常要进行”平滑“，常用”拉普拉斯修正“。令N表示训练集D中可能的类别数，$N_i$表示第i个属性可能的取值数，则公式可以修正为：

$
\hat{P}(c)=\frac{|D_c|+1}{|D|+N} \\
\hat{P}(x_i|c)=\frac{|D_{c,x_i}|+1}{|D_c|+N_i}
$

那么上面的例子可以修正为

$
P(好瓜=是)=\frac{8+1}{17+2}\approx0.474 \\
P(好瓜=否)=\frac{9+1}{17+2}\approx0.526 \\
P(青绿|是)=P(色泽=青绿|好瓜=是)=\frac{3+1}{8+3}\approx0.364 \\
P(青绿|否)=P(色泽=青绿|好瓜=否)=\frac{3+1}{9+3}\approx0.333 \\
$

## 4.2. 使用贝叶斯分类器进行文本分类
《机器学习实战》中基于伯努利模型实现朴素贝叶斯分类器。这种实现方式不考虑词在文档中出现的次数，只考虑出不出现，因此在这个意义上相当于假设词是等权重的。

文档转为词向量的过程：
1. 统计训练集中所有出现过的单词，将单词加入到一个list中（不含重复单词）记为tlist，tlist长度为n
2. 输入一个句子s，并生成一个长度为n的list，如果tlist中位置i的单词可以在s中找到，则相应位置记为1，否则记为0

如此可以得到一个句子向量

In [112]:
def createVocabList(dataSet):
    vocaSet = set([])
    for doc in dataSet:
        vocaSet = vocaSet | set(doc)
    return list(vocaSet)

In [113]:
def myword2vec(vocabList, inputSet):
    returnvec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnvec[vocabList.index(word)]=1
        else:
            print("the word %s is not in my vocabulary!" % word)
    return returnvec

myword2vec(["a","b","c","d"],("a","b","c"))

[1, 1, 1, 0]

In [223]:
import numpy as np
# 从词向量计算概率

'''
trainMatrix : 由训练集组成的句向量矩阵 
labels： 标签，数据类型，dataFrame，column名为"labels"

RETURN：
    p: 一个数组，每个数组是一个向量，表示标签对应向量的概率P(w|c_i)
    P: 一个字典，标明数据集中每个标签类别的概率
    llabelset: 一个数组，有标签集合转化而来，数组中的顺序与p中的值相对应
'''
def trainNB0(trainMatrix, labels):
    # 有n个句子就会有n个向量
    numTrainDocs = len(trainMatrix)
    # 向量的长度
    numWords = len(trainMatrix[0])
    labelset = set(labels.labels.values.tolist())
    N = len(labelset)
    
    # 计算出每种类型所占比例
    P = {}
    llabelset = list(labelset)
    print(llabelset)
    for i in llabelset:
        P[i] = (len(labels[labels.labels==i])+1)/(numTrainDocs+N)
    
    labellist = labels.labels.values.tolist()
    p = {}
    numcate = []
    denom = []
    for i in range(len(llabelset)):
        numcate.append(np.ones(numWords))
        denom.append(2.0)
    
    for j in range(len(llabelset)):
        for i in range(numTrainDocs):
            if labellist[i] == llabelset[j]:
                numcate[j] += trainMatrix[i]
                denom[j] += sum(trainMatrix[i])
                
    pvec = []
    for i in range(len(llabelset)):
        print(numcate[i],denom[i])
        pvec.append(np.log(numcate[i]/denom[i]))
        
    return pvec,P,llabelset



In [224]:
# 西瓜数据集

train = df.iloc[:,0:-1].values
labels = pd.DataFrame(df.iloc[:,-1])
labels[labels.labels=='good']=1
labels[labels.labels=='bad']=0
# labels = np.array(labels)
martix = []

vocalist = createVocabList(train)
print(vocalist)
for i in train:
    martix.append(myword2vec(vocalist, i))

    
trainNB0(martix, labels)

['浊响', '清脆', '凹陷', '稍凹', '平坦', '浅白', '清晰', '稍蜷', '沉闷', '稍糊', '蜷缩', '青绿', '软粘', '硬挺', '硬滑', '乌黑', '模糊']
[0, 1]
[5. 3. 3. 4. 5. 5. 3. 5. 4. 5. 4. 4. 4. 3. 7. 3. 4.] 56.0
[7. 1. 6. 4. 1. 2. 8. 4. 3. 2. 6. 4. 3. 1. 7. 5. 1.] 50.0


([array([-2.41591378, -2.9267394 , -2.9267394 , -2.63905733, -2.41591378,
         -2.41591378, -2.9267394 , -2.41591378, -2.63905733, -2.41591378,
         -2.63905733, -2.63905733, -2.63905733, -2.9267394 , -2.07944154,
         -2.9267394 , -2.63905733]),
  array([-1.96611286, -3.91202301, -2.12026354, -2.52572864, -3.91202301,
         -3.21887582, -1.83258146, -2.52572864, -2.81341072, -3.21887582,
         -2.12026354, -2.52572864, -2.81341072, -3.91202301, -1.96611286,
         -2.30258509, -3.91202301])],
 {0: 0.5263157894736842, 1: 0.47368421052631576},
 [0, 1])

In [225]:
#邮件数据集

martixpostingList = [
    ['my', 'dog', 'has','flea','problems','help','please'],
    ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
    ['my', 'dalmation','is','so','cute','I','love','him'],
    ['stop','posting','stupid','worthless','garbage'],
    ['mr','licks','ate','my','steak','how','to','stop','him'],
    ['quit','buying','worthless','dog','food','stupid']
]
classvec = [0,1,0,1,0,1]

classvec=pd.DataFrame(classvec, columns=['labels'])
vocablist = createVocabList(postingList)
trainMat = []
for i in postingList:
    trainMat.append(myword2vec(vocablist, i))
    
p,P,llabelset = trainNB0(trainMat, classvec)
print(p,P)

[0, 1]
[1. 2. 2. 1. 2. 1. 2. 1. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 1. 1. 2. 2. 1. 2.
 2. 1. 1. 2. 2. 3. 1. 4.] 26.0
[3. 1. 1. 2. 1. 2. 1. 2. 1. 3. 1. 2. 1. 1. 1. 1. 2. 1. 2. 4. 1. 1. 2. 2.
 1. 2. 2. 1. 1. 2. 2. 1.] 21.0
[array([-3.25809654, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
       -3.25809654, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -2.56494936, -2.56494936, -2.56494936, -3.25809654, -3.25809654,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -3.25809654, -3.25809654, -2.56494936, -2.56494936, -2.15948425,
       -3.25809654, -1.87180218]), array([-1.94591015, -3.04452244, -3.04452244, -2.35137526, -3.04452244,
       -2.35137526, -3.04452244, -2.35137526, -3.04452244, -1.94591015,
       -3.04452244, -2.35137526, -3.04452244, -3.04452244, -3.04452244,
       -3.04452244, -2.35137526, -3.04452244, -2.35137526, -1.65822808,
       -3.04452244, -3.044522

In [226]:
# 分类函数
'''
vec2Classfied: 等待分类的向量
p: 一个数组，每个数组是一个向量，表示标签对应向量的概率P(w|c_i)
P: 一个字典，标明数据集中每个标签类别的概率
llabelset: 一个数组，有标签集合转化而来，数组中的顺序与p中的值相对应

RETURN:
    llabelset[index] 类别
'''

def classfyNB(vec2Classfied, p, P,llabelset):
    result = []
    for i in range(len(p)):
        print(llabelset[i])
        result.append(sum(vec2Classfied*p[i])+np.log(P[llabelset[i]]))
        print(result[i])
    maxnum = max(result)
    index = result.index(maxnum)
    return llabelset[index]
    
    
test = ["love",'my','dalmation']
test2 = ['stupid','garbage']
thisDoc = np.array(myword2vec(vocablist, test))
print(classfyNB(thisDoc,p,P,llabelset))
print("-------------------------------")
thisDoc = np.array(myword2vec(vocablist, test2))
print(classfyNB(thisDoc,p,P,llabelset))

0
-7.694848072384611
1
-9.826714493730215
0
-------------------------------
0
-7.20934025660291
1
-4.702750514326955
1
